In [180]:
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model
import random
import json
import pickle
import numpy as np

In [151]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

In [152]:
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
context = pickle.load(open('context.pkl','rb'))
model = load_model('chat_bot.keras')

In [153]:
def clean_sentences(sentences):
    sentences_words = nltk.word_tokenize(sentences)
    sentences_words = [lemmatizer.lemmatize(word.lower()) for word in sentences_words]
    return sentences_words

In [154]:
def bag_of_words(sentences,words,show_details=True):
    # call clean_sentences to toennize the sentences
    sentence_words = clean_sentences(sentences)
    #bag of words from the sentences
    bag = [0]*len(words)
    for sent in sentence_words:
        for index , word in enumerate(words):
            if word == sent:
                bag[index] = 1
                if show_details:
                    print("Found in Bag of Words : %s"% word)

    return(np.array(bag))

In [155]:
def predict(sentences,model):
    # Creazte a threshhold filter 
    p = bag_of_words(sentences,words,show_details=False)
    res = model.predict(np.array([p]))[0]
    threshold = 0.25

    results = [[index, result] for index, result in enumerate(res) if result > threshold]


    # softmax gies probability scores well sort by that
    results.sort(key = lambda x : x[1], reverse=True)

    return_list =  []
    for r in results:
        return_list.append({"intent" : classes[r[0]], "probability":str(r[1])})
    return return_list

In [159]:
def getResponse(ints,intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break

    return result

In [160]:
def chatbot_response(message):
    ints = predict(message,model)
    res = getResponse(ints,intents)
    
    return res



In [179]:

#Creating GUI with tkinter
import tkinter
from tkinter import *
from PIL import ImageTk,Image

def send(event=None):
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n','right_align')
        ChatLog.tag_configure('right_align', justify='right')
        ChatLog.config(foreground="#442265", font=("Times New Roman", 12 ))
    
        res = chatbot_response(msg)
        ChatLog.insert(END, "Sahayak: " + res + '\n\n', 'left_align')
        ChatLog.tag_configure('left_align', justify='left')            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)



def on_entry_click(event):
    if EntryBox.get("1.0", "end-1c") == "Type here":
        EntryBox.delete("1.0", "end")
        EntryBox.insert("1.0", "")
        EntryBox.config(fg='black')

def on_focusout(event):
    if not EntryBox.get("1.0", "end-1c"):
        EntryBox.insert("1.0", "Type here")
        EntryBox.config(fg='grey')
def on_key_press(event):
    if EntryBox.get("1.0", "end-1c") == "Type here":
        EntryBox.delete("1.0", "end")
        EntryBox.config(fg='black')


def clear_conversations():
    ChatLog.config(state=NORMAL)
    ChatLog.delete("1.0", END)
    ChatLog.config(state=DISABLED)
 



base = Tk()
base.title("Sahayak:     ")
base.geometry("450x500")
#base.resizable(width=650, height=420)
base.minsize(430,300) 
base.maxsize(400,500) 

base.iconbitmap("favicon_test.ico")

base.configure(background='#B09FCA')



img_new = Image.open('sahayak2.png')  
resized_img_new = img_new.resize((400, 80))
img_new = ImageTk.PhotoImage(resized_img_new)
img_label_new = Label(base, image=img_new)
img_label_new.pack( padx=(5,5),pady=(5,5),anchor='nw',side='top')


ChatLog = Text(base, bd=3, bg="#E6E6FA", height="4", width="20", font="Arial",)
ChatLog.config(state=DISABLED)


#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="arrow",background='#E6E6FA',troughcolor='#E6E6FA')
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Ask", width='5', height='4',
                    bd=3, bg="#967BB6",
                    command= send )


# Clear Button
ClearButton = Button(base, font=("Verdana", 12, 'bold'), text="Clear", width='5', height='4',
                    bd=3, bg="#967BB6",
                    command=clear_conversations)


#Create the box to enter message
EntryBox = Text(base, bd=3, bg="#E6E6FA",width="45", height="5", font="Helentica",)
EntryBox.bind("<FocusIn>", on_entry_click)
EntryBox.bind("<Key>", on_key_press)
EntryBox.bind("<FocusOut>", on_focusout)
EntryBox.insert("1.0", "")
EntryBox.config(fg='black')
EntryBox.bind("<Return>", send)




#Place all components on the screen
scrollbar.place(x=410,y=6, height=386)
ChatLog.place(x=1,y=80, height=320, width=410)
EntryBox.place(x=1, y=401, height=90, width=350)
SendButton.place(x=360, y=401, height=45)
ClearButton.place(x=360, y=450, height=45)

base.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
